# Parse the json files that have been saved to disk

In [1]:
import pandas as pd
import uuid
import json
import os
import uuid
import re

from datetime import datetime

In [2]:
fpath = "D://projects//_external_files//reliefweb_situation_reports//"
output_fpath = "D://projects//_external_files//surveyor//rw_siturep_preprocessed//"

In [3]:
def extract_themes(j):
    #must pass in after levelling json up to j['fields']
    themes = j.get('theme')
    if themes is None:
        return None
        
    ts =[]
    for theme in themes:
        ts.append(theme['name'].lower())
    return '; '.join(ts)


def parse_sitrep_json(j):
    #for j in json_data:
    
    #parse 
    try:
        reference_url = j['href']
            
        j = j['fields']
        #print(j)
        
        glide_id = None
        disaster = j.get('disaster')
        if disaster:
            glide_id = disaster[0]['glide']
    
        
        rec_id = j['id']
        title = j['title']
        original_text = j['body']
        link_to_doc = j['url_alias']
        file_url = j['file'][0]['url']
        primary_country_iso3 = j['primary_country']['iso3']
        primary_country = j['primary_country']['shortname']
        author_org = j['source'][0]['shortname']
        report_date = j['date']['original']
        themes = extract_themes(j)
       
        original_text_list = original_text.split("\n\n")
        idx_para = 0
        for o in original_text_list:
            row = ['situation report',reference_url,glide_id,idx_para,primary_country,title,themes,o,file_url,o,author_org,report_date]
            df_reliefweb_situation_report.loc[len(df_reliefweb_situation_report)] = row
            idx_para += 1
    except:
        pass

In [4]:
#prep receiving df
df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','idx_para','source_level_country','source_title','source_desc',
                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])


for f in os.listdir(fpath):
    with open(f"{fpath}{f}", 'r') as file:
        json_data = json.load(file)
        parse_sitrep_json(json_data)

In [5]:
df_reliefweb_situation_report['para_id'] = 'rwsitrep_' + df_reliefweb_situation_report['reference_url'] + '_' + df_reliefweb_situation_report['idx_para'].astype(str)
#df_reliefweb_situation_report = df_reliefweb_situation_report[df_reliefweb_situation_report['glide_id'] == 'EQ-2023-000015-TUR']
#df_reliefweb_situation_report

In [9]:
def generate_uuid(x):
    return uuid.uuid4().hex
    
def undesirable_char_remover(text, ignore=["." , ",", " ", ":", "-", "_", "(", ")", ";", "/", "'"]):
    return_chars = []
    
    def screen_char(char):
        if c in ignore:
            return_chars.append(c)
        if c.isalnum() == True:
            return_chars.append(c)
        return None

    if isinstance(text, str):
        for c in text:
            screened = screen_char(c)
            if screened is not None:
                return_chars.append(c)
    else:
        print("must pass in a string to this function")
        return text
    

    return ''.join(return_chars).strip() 

df_reliefweb_situation_report['source_original_text'] = df_reliefweb_situation_report['source_original_text'].apply(undesirable_char_remover)
df_reliefweb_situation_report['text'] = df_reliefweb_situation_report['text'].apply(undesirable_char_remover)

output_file = f"D://projects//_external_files//surveyor//rw_sitrep_preprocessed//disaster_sitrep_json_parsed_{generate_uuid(1)}.xlsx"
df_reliefweb_situation_report.to_excel(output_file, sheet_name="json_parse", index=False)
print(output_file)

D://projects//_external_files//surveyor//rw_sitrep_preprocessed//disaster_sitrep_json_parsed_bdfc9ddf9d3b40eba3f11669a3619268.xlsx
